In [31]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import os
from PIL import Image
import torch.nn.functional as F

In [32]:
base_dir = 'img'

In [33]:
def calculate_cosine_score(original_image, modified_image):
    # Convert the images to PyTorch tensors
    original_tensor = torch.tensor(original_image, dtype=torch.float32)
    modified_tensor = torch.tensor(modified_image, dtype=torch.float32)

    # Flatten the tensors to represent the images as vectors
    original_vector = original_tensor.flatten()
    modified_vector = modified_tensor.flatten()

    # Ensure the vectors have the same size
    max_size = max(original_vector.size(0), modified_vector.size(0))
    original_vector = F.pad(original_vector, (0, max_size - original_vector.size(0)), mode='constant', value=0)
    modified_vector = F.pad(modified_vector, (0, max_size - modified_vector.size(0)), mode='constant', value=0)

    # Calculate the cosine similarity
    cosine_similarity = F.cosine_similarity(original_vector.unsqueeze(0), modified_vector.unsqueeze(0), dim=1)

    return cosine_similarity.item()

In [34]:
# Set the base directory
base_dir = 'img'

# Get the list of image files in the 'original' folder
original_folder = os.path.join(base_dir, 'original')
original_files = os.listdir(original_folder)

# Loop through the image files
for original_file in original_files:
    # Construct the full file paths
    original_path = os.path.join(original_folder, original_file)
    modified_path = os.path.join(os.path.join(base_dir, 'modified'), original_file)

    # Load the images
    original_image = np.array(Image.open(original_path))
    modified_image = np.array(Image.open(modified_path))

    # Resize the smaller image to match the larger one
    original_size = original_image.shape[:2]
    modified_size = modified_image.shape[:2]
    if original_size != modified_size:
        if original_size[0] > modified_size[0] or original_size[1] > modified_size[1]:
            modified_image = np.array(Image.fromarray(modified_image).resize(original_size, resample=Image.BICUBIC))
        else:
            original_image = np.array(Image.fromarray(original_image).resize(modified_size, resample=Image.BICUBIC))

    # Calculate the cosine similarity score
    cosine_score = calculate_cosine_score(original_image, modified_image)
    print(f"Cosine similarity score for {original_file}: {cosine_score}")

Cosine similarity score for 1.png: 0.8999110460281372
Cosine similarity score for 2.png: 0.7618823647499084
Cosine similarity score for 3.png: 0.7268241047859192
